In [ ]:
import os
import subprocess

# Set up FSL environment
fsl_path = "/usr/local/fsl"
os.environ["FSLDIR"] = fsl_path
os.environ["PATH"] += os.pathsep + os.path.join(fsl_path, "bin")
os.environ["FSLOUTPUTTYPE"] = "NIFTI_GZ"

# Define base directory where the BIDS data is stored
BASE_DIR = "/Volumes/datadrive/test/forkx/MDD_nii"

# Path to the list of subjects file
SUBJECT_LIST = "/Users/gengshuang/Desktop/depression/code/lists.txt"

# Define constants for acqp and index files
ACQP = "/Users/gengshuang/Desktop/depression/code/acqparams.txt"
INDEX = "/Users/gengshuang/Desktop/depression/code/index.txt"

# Read the list of subjects
with open(SUBJECT_LIST, 'r') as file:
    subjects = file.read().splitlines()

# Loop over each subject in the list
for SUBJ_ID in subjects:
    SUBJ_ID = f"sub-{SUBJ_ID}"
    print(f"Processing {SUBJ_ID}")

    # Define subject directory
    SUBJ_DIR = os.path.join(BASE_DIR, SUBJ_ID)

    # Define file paths
    T1w_IMAGE = os.path.join(SUBJ_DIR, 'anat', f"{SUBJ_ID}_T1w.nii.gz")
    DWI_IMAGE = os.path.join(SUBJ_DIR, 'dwi', f"{SUBJ_ID}_dwi.nii.gz")
    MASK = os.path.join(SUBJ_DIR, 'anat', f"{SUBJ_ID}_T1w_brain-mask.nii.gz")
    RESAMPLED_MASK = os.path.join(SUBJ_DIR, 'dwi', f"{SUBJ_ID}_dwi-resam.nii.gz")
    BVEC = os.path.join(SUBJ_DIR, 'dwi', f"{SUBJ_ID}_dwi.bvec")
    BVAL = os.path.join(SUBJ_DIR, 'dwi', f"{SUBJ_ID}_dwi.bval")
    OUTPUT = os.path.join(SUBJ_DIR, 'dwi', f"{SUBJ_ID}_eddy_corrected")

    # Check if the T1w image exists
    if not os.path.exists(T1w_IMAGE):
        print(f"Error: {T1w_IMAGE} does not exist.")
        continue

    # Run BET to extract brain mask
    bet_cmd = ['bet', T1w_IMAGE, MASK, '-f', '0.2']
    try:
        subprocess.run(bet_cmd, check=True)
    except subprocess.CalledProcessError as e:
        print(f"Error running BET: {e}")
        continue

    # Check if the MASK file was created
    if not os.path.exists(MASK):
        print(f"Error: {MASK} was not created.")
        continue

    # Run FLIRT to resample mask to DWI space
    flirt_cmd = ['flirt', '-in', MASK, '-ref', DWI_IMAGE, '-out', RESAMPLED_MASK, '-applyxfm', '-interp', 'nearestneighbour']
    try:
        subprocess.run(flirt_cmd, check=True)
    except subprocess.CalledProcessError as e:
        print(f"Error running FLIRT: {e}")
        continue

    # Check if the RESAMPLED_MASK file was created
    if not os.path.exists(RESAMPLED_MASK):
        print(f"Error: {RESAMPLED_MASK} was not created.")
        continue

    # Run EDdy to correct distortions
    eddy_cmd = [
        'eddy',
        f'--imain={DWI_IMAGE}',
        f'--mask={RESAMPLED_MASK}',
        f'--acqp={ACQP}',
        f'--index={INDEX}',
        f'--bvecs={BVEC}',
        f'--bvals={BVAL}',
        '--repol',
        f'--out={OUTPUT}'
    ]
    try:
        subprocess.run(eddy_cmd, check=True)
    except subprocess.CalledProcessError as e:
        print(f"Error running EDdy: {e}")
        continue

    print(f"{SUBJ_ID} processing completed.")


Processing sub-060



eddy diffusion --imain='my_ima' --acqp='my_acqp' ...


sub-060 processing completed.
Processing sub-065



eddy diffusion --imain='my_ima' --acqp='my_acqp' ...


sub-065 processing completed.
Processing sub-075



eddy diffusion --imain='my_ima' --acqp='my_acqp' ...


